# Содержание #
<div>
  <ol>
    <li>
        <a href="#project">Прогнозирование заказов такси</a>
    </li>
    <li>
        <a href="#import_libs">Импорт библиотек</a>
    </li>
    <li>
        <a href="#data">Подготовка</a>
    </li>
        <ul>
            <li>
                <a href="#data_load">Загрузка данных</a>
            </li>
            <li>
                <a href="#data_preview">Осмотр данных</a>
            </li>
            <li>
                <a href="#data_datetime">Приведение типа данных</a>
            </li>
            <li>
                <a href="#data_sort">Сортировка данных в хронологическом порядке</a>
            </li>
        </ul>
      <li>
          <a href="#analyze">Анализ</a>
      </li>
      <ul>
          <li>
              <a href="#analyze_resample">Ресемплирование данных</a>
          </li>
      </ul>
      <li>
          <a href="#study">Обучение моделей</a>
      </li>
      <ul>
          <li>
              <a href="#study_const_features">Формирование постоянных признаков</a>
          </li>
          <li>
              <a href="#study_features_function">Функция формирования признаков</a>
          </li>
          <li>
              <a href="#study_rmse_function">Функция расчета значения метрики RMSE</a>
          </li>
          <li>
              <a href="#study_train_test_func">Функция формирование учебного/тестового наборов признаков, ответов</a>
          </li>
          <li>
              <a href="#study_scorer_init">Инициализация оценщика</a>
          </li>
          <li>
              <a href="#sudy_lag_rolling">Формирование данных с максимальным отставанием, размером скользящего окна</a>
          </li>
          <li>
              <a href="#study_train_test">Деление данных на учебный/тестовый наборы</a>
          </li>
          <li>
              <a href="#study_cat_boost_params">Подбор гиперпараметров CatBoostRegressor</a>
          </li>
          <li>
              <a href="#study_dec_tree">Подбор гиперпараметров DecisionTreeRegressor</a>
          </li>
      </ul>
      <li>
          <a href="#test">Тестирование</a>
      </li>
      <ul>
          <li>
              <a href="#test_cat_boost">Тестирование модели "CatBoostRegressor"</a>
          </li>
          <li>
              <a href="#test_lin_regres">Тестирование модели "LinearRegression"</a>
          </li>
          <li>
              <a href="#test_dec_tree_regres">Тестирование модели "DecisionTreeRegressor"</a>
          </li>
      </ul>
      <li>
          <a href="#output">Вывод</a>
      </li>
  </ol>
</div>

<a id='project'></a>
# Прогнозирование заказов такси #

<a id='import_libs'></a>
# Импорт библиотек #

In [22]:
import pandas as pd
from IPython.core.display import display
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
state = 80

<a id='data'></a>
# Подготовка #

<a id='data_load'></a>
## Загрузка данных ##

In [23]:
data = pd.read_csv('datasets/taxi.csv')

<a id='data_preview'></a>
## Осмотр данных ##

In [24]:
display(data)

,datetime,num_orders
0,2018-03-01 00:00:00,9
1,2018-03-01 00:10:00,14
2,2018-03-01 00:20:00,28
3,2018-03-01 00:30:00,20
4,2018-03-01 00:40:00,32
...,...,...
26491,2018-08-31 23:10:00,32
26492,2018-08-31 23:20:00,24
26493,2018-08-31 23:30:00,27
26494,2018-08-31 23:40:00,39


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   datetime    26496 non-null  object
 1   num_orders  26496 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 414.1+ KB


## Результаты осмотра данных ##
1. пропущенные значения отсутствуют
2. data['datetime'] - привести к типу date

<a id='data_datetime'></a>
## Приведение типа данных ##

In [26]:
data['datetime'] = pd.to_datetime(data['datetime'])
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    26496 non-null  datetime64[ns]
 1   num_orders  26496 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 414.1 KB


None

<a id='data_sort'></a>
## Сортировка данных в хронологическом порядке ##

In [27]:
data = data.set_index('datetime')
data.sort_index(inplace=True)
display(data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


None

In [28]:
display(data.index.is_monotonic)

True

## Результаты подготовки данных ##
1. data['datetime'] приведен к типу datetime
2. data['datetime'] отсортированы в порядке возрастания

<a id='analyze'></a>
# Анализ #
1. В задаче указана необходимость прогноза количества заказов такси на следующий час =>
необходимо изменить временной интервал, в котором учитывается количество заказов такси до 1 часа.

<a id='analyze_resample'></a>
## Ресемплирование данных ##

In [ ]:
data = data.resample('1H').sum()
display(data)

<a id='study'></a>
# Обучение моделей #

<a id='study_const_features'></a>
## Формирование постоянных признаков ##

In [30]:
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['dayofweek'] = data.index.dayofweek
data['hour'] = data.index.hour

<a id='study_features_function'></a>
## Функция формирования признаков ##

In [31]:
def make_features_func(data, max_lag, rolling_mean_size):
    make_data = data.copy()

    for lag in range(1, max_lag + 1):
        make_data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    make_data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()
    return make_data

<a id='study_rmse_function'></a>
## Функция расчета значения метрики RMSE ##

In [32]:
def rmse_func(target_real, target_predict):
    rmse = abs(math.sqrt(mean_squared_error(y_true=target_real,
                                        y_pred=target_predict)))
    return rmse

<a id='study_train_test_func'></a>
## Функция формирование учебного/тестового наборов признаков, ответов ##

In [33]:
def get_train_test_func(make_data):
    make_data = make_data.dropna()
    train, test = train_test_split(make_data,
                                   shuffle=False,
                                   test_size=0.1)
    features_train = train.drop(columns='num_orders')
    target_train = train['num_orders']

    features_test = test.drop(columns='num_orders')
    target_test = test['num_orders']
    return features_train, target_train, features_test, target_test

<a id='study_scorer_init'></a>
## Инициализация оценщика ##

In [34]:
scorer = make_scorer(score_func=rmse_func,
                     greater_is_better=False)

<a id='sudy_lag_rolling'></a>
## Формирование данных с максимальным отставанием, размером скользящего окна ##

In [35]:
make_data = make_features_func(data=data, max_lag=1, rolling_mean_size=1)

<a id='study_train_test'></a>
## Деление данных на учебный/тестовый наборы ##

In [36]:
features_train, target_train, features_test, target_test = get_train_test_func(make_data=data)

<a id='study_cat_boost_params'></a>
## Подбор гиперпараметров CatBoostRegressor ##

In [37]:
params = {'depth'                 : [4, 7, 10],
          'learning_rate'         : [0.03, 0.1, 0.15],
          'l2_leaf_reg'           : [1,4,9],
          'iterations'            : [300]}

tscv = TimeSeriesSplit(n_splits=5)

clf = CatBoostRegressor()

cbr_model_params = clf.grid_search(param_grid=params,
                                   X=features_train,
                                   y=target_train,
                                   cv=tscv)

0:	learn: 77.0976629	test: 112.3186240	best: 112.3186240 (0)	total: 675us	remaining: 202ms
1:	learn: 75.2047768	test: 110.3584620	best: 110.3584620 (1)	total: 1.98ms	remaining: 295ms
2:	learn: 73.4175152	test: 108.4994425	best: 108.4994425 (2)	total: 3.24ms	remaining: 321ms
3:	learn: 71.6711323	test: 106.1975219	best: 106.1975219 (3)	total: 4.24ms	remaining: 314ms
4:	learn: 69.8866963	test: 104.1246292	best: 104.1246292 (4)	total: 5.31ms	remaining: 314ms
5:	learn: 68.2346917	test: 102.0783719	best: 102.0783719 (5)	total: 6.52ms	remaining: 319ms
6:	learn: 66.5609058	test: 100.3511443	best: 100.3511443 (6)	total: 7.42ms	remaining: 311ms
7:	learn: 64.9677843	test: 98.6948041	best: 98.6948041 (7)	total: 8.4ms	remaining: 307ms
8:	learn: 63.4974413	test: 96.8244335	best: 96.8244335 (8)	total: 9.44ms	remaining: 305ms
9:	learn: 62.0799746	test: 95.0121393	best: 95.0121393 (9)	total: 10.5ms	remaining: 303ms
10:	learn: 60.7157883	test: 93.2657047	best: 93.2657047 (10)	total: 11.5ms	remaining: 30

<a id='study_dec_tree'></a>
## Подбор гиперпараметров DecisionTreeRegressor ##

In [38]:
params = {'splitter'              : ['best', 'random'],
          'max_depth'             : range(1, 21, 4),
          'min_samples_split'     : range(2,21,4),
          'min_samples_leaf'      : range(1,21,4),
          'max_features'          : ['auto']}

tscv = TimeSeriesSplit(n_splits=5)

clf = DecisionTreeRegressor()
grid = GridSearchCV(clf,
                    param_grid=params,
                    scoring=scorer,
                    cv=tscv)
grid.fit(X=features_train,
         y=target_train)

dec_tree_model_params = grid.best_params_

<a id='test'><a/>
# Тестирование #

<a id='test_cat_boost'></a>
## Тестирование модели "CatBoostRegressor" ##

In [39]:
clf = CatBoostRegressor()
clf.set_params(**cbr_model_params.get('params'))
clf.fit(X=features_train, y=target_train)
score = rmse_func(target_real=target_test,
                   target_predict=clf.predict(data=features_test))
result = pd.DataFrame({'Модель' : type(clf).__name__,
                       'RMSE score' : score}, index=[0])

0:	learn: 36.2701034	total: 1ms	remaining: 300ms
1:	learn: 34.2431477	total: 2.28ms	remaining: 340ms
2:	learn: 32.9152483	total: 3.28ms	remaining: 325ms
3:	learn: 31.5599179	total: 4.31ms	remaining: 319ms
4:	learn: 30.5466895	total: 5.33ms	remaining: 315ms
5:	learn: 29.6921166	total: 6.27ms	remaining: 307ms
6:	learn: 28.9787833	total: 7.31ms	remaining: 306ms
7:	learn: 28.2832364	total: 8.22ms	remaining: 300ms
8:	learn: 27.8458489	total: 9.14ms	remaining: 296ms
9:	learn: 27.5002741	total: 10ms	remaining: 290ms
10:	learn: 27.2133030	total: 10.8ms	remaining: 285ms
11:	learn: 26.7494982	total: 11.8ms	remaining: 282ms
12:	learn: 26.5175877	total: 12.7ms	remaining: 279ms
13:	learn: 26.2898263	total: 13.6ms	remaining: 278ms
14:	learn: 26.1007566	total: 14.6ms	remaining: 276ms
15:	learn: 25.8137126	total: 15.4ms	remaining: 273ms
16:	learn: 25.6770103	total: 16.3ms	remaining: 271ms
17:	learn: 25.5392421	total: 17.2ms	remaining: 269ms
18:	learn: 25.4387195	total: 18.1ms	remaining: 267ms
19:	lear

<a id='test_lin_regres'></a>
## Тестирование модели "LinearRegression" ##

In [40]:
clf = LinearRegression()
clf.fit(X=features_train, y=target_train)

score = rmse_func(target_real=target_test,
                  target_predict=clf.predict(X=features_test))

result = result.append({'Модель' : type(clf).__name__,
                       'RMSE score' : score}, ignore_index=True)

<a id='test_dec_tree_regres'></a>
## Тестирование модели "DecisionTreeRegressor" ##

In [41]:
clf = DecisionTreeRegressor()
clf.set_params(**dec_tree_model_params)
clf.fit(X=features_train,
        y=target_train)

score = rmse_func(target_real=target_test,
                  target_predict=clf.predict(X=features_test))

result = result.append({'Модель' : type(clf).__name__,
                       'RMSE score' : score}, ignore_index=True)

In [42]:
display(result)

,Модель,RMSE score
0,CatBoostRegressor,46.730586
1,LinearRegression,63.789001
2,DecisionTreeRegressor,59.576707


<a id='output'></a>
# Вывод #
1. Значение метрики RMSE < 48 достигнуто для модели "CatBoostRegressor", при:
    - максимальный размер отставания = 1 часу
    - размер окна для скользящего среднего = 1 часу